#Introduciton
My name is Jake and I am 14. I spent the last weeks of my summer break learning how to create custom TensorFlow object classifiers on my Google AIY Vision kit. It is very hard to get started with Deep Learning so I wanted to simplify things and make a very straightforward tutorial on Image Classification on the Google AIY Vision Kit.

Due to the Vision Kit's little amount of GPU memory we won't be able to do Single Shot MultiBox Detection (where each object has a box around it), but image classification instead. Image Classification is similar to Object Detection but instead of looking for all objects and highlighting where they are, it classifies images into a single category that correspond to the most salient object.


![Image Classification vs. Object Detection](https://cdn-images-1.medium.com/max/1600/1*Hz6t-tokG1niaUfmcysusw.jpeg)


If you have any questions, don't hesitate to email me: jakebmalis@gmail.com

#Setup

First we start by **importing os**, which is a library that allows us to do local server commands such as switching directories, and importing **files** which allows the server to communicate with your local desktop.

Next we download Google's Tensorflow for Poets tutorial with **git clone**, which contains the code necesary to train and test the model.

Then the commands rename the directory, and enter it.

After we have downloaded the code, we use **wget** to download the training images from my AWS bucket.

The file that we just downloaded is a **.zip** file which needs to be decompressed by running the **unzip** command.


In [ ]:
import os
from google.colab import files


!git clone https://github.com/googlecodelabs/tensorflow-for-poets-2.git
!mv tensorflow-for-poets-2 tensorflow
os.chdir('tensorflow')

!wget https://s3.amazonaws.com/tensorflow-deeplearning/tf_pictures.zip -P tf_files/
!unzip tf_files/tf_pictures.zip -d tf_files/

Cloning into 'tensorflow-for-poets-2'...
remote: Enumerating objects: 426, done.
remote: Total 426 (delta 0), reused 0 (delta 0), pack-reused 426
Receiving objects: 100% (426/426), 33.97 MiB | 37.28 MiB/s, done.
Resolving deltas: 100% (159/159), done.
--2019-07-10 18:51:40--  https://s3.amazonaws.com/tensorflow-deeplearning/tf_pictures.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.147.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.147.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433713428 (414M) [application/zip]
Saving to: ‘tf_files/tf_pictures.zip’

tf_pictures.zip     100%[===================>] 413.62M  74.9MB/s    in 6.3s    

2019-07-10 18:51:47 (65.6 MB/s) - ‘tf_files/tf_pictures.zip’ saved [433713428/433713428]

Archive:  tf_files/tf_pictures.zip
   creating: tf_files/tf_pictures/
   creating: tf_files/tf_pictures/king_of_spades/
  inflating: tf_files/tf_pictures/king_of_spades/20180722_095900.jpg  
  inflating: tf_files/t

#Train
Now we are going to run a python retraining script for a MobileNet model.

The code specifies the name of the output files, the pre-trained model type, and the directory of where the pictures are stored.

In [ ]:
!python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --model_dir=tf_files/models/"mobilenet_1.0_192" \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture="mobilenet_1.0_192" \
  --learning_rate=0.005 \
  --image_dir=tf_files/tf_pictures

W0710 18:52:06.795366 140357633222528 deprecation_wrapper.py:119] From /content/tensorflow/scripts/retrain.py:1326: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0710 18:52:06.795884 140357633222528 deprecation_wrapper.py:119] From /content/tensorflow/scripts/retrain.py:970: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0710 18:52:06.796011 140357633222528 deprecation_wrapper.py:119] From /content/tensorflow/scripts/retrain.py:970: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0710 18:52:06.796115 140357633222528 deprecation_wrapper.py:119] From /content/tensorflow/scripts/retrain.py:835: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0710 18:52:06.796275 140357633222528 deprecation_wrapper.py:119] From /content/tensorflow/scripts/retrain.py:837: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs i

#Compile
Now we are going to download Google's AIY Vision kit compiler with the **wget** command, and use the **tar xzvf** command to turn Google's tar file into a usable **par** file.

Now that we have this usable par file, we can run it with the retrained graph as the input.

In [ ]:
os.chdir('tf_files')
!wget https://dl.google.com/dl/aiyprojects/vision/bonnet_model_compiler_latest.tgz
!tar xzvf bonnet_model_compiler_latest.tgz

!./bonnet_model_compiler.par \
  --frozen_graph_path=retrained_graph.pb \
  --output_graph_path=retrained_graph.binaryproto \
  --input_tensor_name=input \
  --output_tensor_names=final_result \
  --input_tensor_size=192

--2019-07-10 18:56:17--  https://dl.google.com/dl/aiyprojects/vision/bonnet_model_compiler_latest.tgz
Resolving dl.google.com (dl.google.com)... 173.194.197.91, 173.194.197.136, 173.194.197.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.197.91|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.google.com/dl/aiyprojects/vision/bonnet_model_compiler_2018_04_26.tgz [following]
--2019-07-10 18:56:17--  https://dl.google.com/dl/aiyprojects/vision/bonnet_model_compiler_2018_04_26.tgz
Reusing existing connection to dl.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 5058328 (4.8M) [application/x-gtar]
Saving to: ‘bonnet_model_compiler_latest.tgz’

bonnet_model_compil 100%[===================>]   4.82M  14.0MB/s    in 0.3s    

2019-07-10 18:56:18 (14.0 MB/s) - ‘bonnet_model_compiler_latest.tgz’ saved [5058328/5058328]

bonnet_model_compiler.par
VisionBonnet model binary retrained_graph.binaryproto generated.


#Download
Lastly we will run two commands that will download both of the files that need to be put on the Vision kit for our custom model to run.

In [ ]:
files.download('retrained_graph.binaryproto')
files.download('retrained_labels.txt')

#Next Steps

###Upload files to Vision Kit and Run
Upload the **.binaryproto** and **.txt** files to the home directory of the Pi (/home/pi) using your preferred FPT software (I prefer FileZilla). After you have uploaded the files to the correct directory you can then run the following code on the pi to start the classification:



```
~/AIY-projects-python/src/examples/vision/mobilenet_based_classifier.py \
  --model_path ~/retrained_graph.binaryproto \
  --label_path ~/retrained_labels.txt \
  --input_height 192 \
  --input_width 192 \
  --input_layer input \
  --output_layer final_result \
  --preview
```



###Run your own custom Mobilenet Classifier
To create your own custom Mobilenet Classifier, we're going to follow a few steps to modify the Colab.

First, create and enter a folder on your local computer named **tf_pictures**.

Then create the directories. If you wanted to create a classfier that could detect 3 items (shirt, pants, and shoes), create a folder for each of those items.

Inside each folder put **at least** 50 images of each object.

Then zip the folder and name it **tf_pictures.zip**.

Finally, upload the .zip file to the file uploader holder of your choice (i.e. Google Cloud Storage Buckets, AWS S3, or Dropbox (this being the most simple)), and change the following lines in the **Setup** section from:


```
!mv tensorflow-for-poets-2 tensorflow
os.chdir('tensorflow')

!wget https://s3.amazonaws.com/tensorflow-deeplearning/tf_pictures.zip -P tf_files/
```
To this:


```
!mv tensorflow-for-poets-2 tensorflow_custom
os.chdir('tensorflow_custom')

!wget https://your.file.location.com/tf_pictures.zip -P tf_files/
```
And repeat the entire process.

